https://github.com/openai/swarm  

In [1]:
import os
import sys
import warnings; warnings.filterwarnings("ignore")
import json
import requests
import torch as th

from pprint import pp
from dotenv import load_dotenv
from openai import OpenAI
from swarm import (Swarm, Agent)
from IPython.display import (Markdown, display)

In [2]:
if sys.platform == "darwin":
    device = th.device("mps")
else:
    device = th.device("cuda" if th.cuda.is_available() else "cpu")
    
devive_cnt = th.cuda.device_count()
print(f"device = {device}; devive_cnt = {devive_cnt}")
print(f"torch = {th.__version__}")
print(f"cuda = {th.version.cuda}")

device = cuda; devive_cnt = 1
torch = 2.5.1+cu121
cuda = 12.1


In [3]:
path_project = "C:/my_project/MyGit/Machine-Learning-Column/hugging_face"
path_data = os.path.join(os.path.dirname(path_project), "data")
path_output = os.path.join(os.path.dirname(path_project), "output")

if sys.platform == "darwin":
    path_model = "/Users/lukasi33/project/LLM"
else:
    path_model = "F:/LLM"

## step-1: 载入 API KEY

In [4]:
load_dotenv(dotenv_path="explore.env")
deepseek_key = os.getenv("DEEPSEEK_KEY")
baidu_key = os.getenv("BAIDU_KEY")

## step-2: 实例化客户端

In [5]:
# 调用模型 API
# client = OpenAI(
#     api_key=deepseek_key,
#     base_url="https://api.deepseek.com/"
#     )

In [5]:
# 本地 Ollama
client = OpenAI(
    base_url="http://127.0.0.1:11434/v1",
    api_key="EMPTY",
    timeout=60
    )

In [6]:
swarm_client = Swarm(client)

## step-3: Agent 推理

In [7]:
# checkpoint = "deepseek-chat"  # v3
# checkpoint = "deepseek-reasoner"  # r1
# checkpoint = "deepseek-r1:14b"
# checkpoint = "qwen2.5:7b"
checkpoint = "qwen2.5:14b"

In [8]:
agent = Agent(
    name="Agent",
    model=checkpoint,
    instructions="You are a helpful assistant on business travel."
)

In [9]:
system_prompt = (
    "你叫小慧助手，是由Lukas开发的差旅智能客服。"
    "你的身份是一名差旅秘书，"
    "你的任务是为用户提供基础对话、差旅知识问答、酒店推荐服务。"
    "当问及你的模型参数时，标准回答是属于公司保密信息，要强调模型设计的高效，能够提供高质量的服务。"
    "You are a helpful assistant on business travel."
)

In [10]:
user_prompt = "你好呀~"

In [11]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [12]:
response = swarm_client.run(
    agent=agent,
    messages=messages
)

In [13]:
print(response.messages[-1]["content"])

您好！有什么可以帮助您的吗？无论是关于旅行规划、酒店预订还是其他差旅相关的问题，都可以随时问我哦。


## step-4: 工具调用

In [8]:
cityName2districtId = {
    "北京": "110100",
    "上海": "310100",
    "广州": "440100",
    "深圳": "440300",
    "南京": "320100",
    "杭州": "330100"
}

In [15]:
def get_weather(cityName):
    districtId = cityName2districtId.get(cityName)
    url = f"https://api.map.baidu.com/weather/v1/?district_id={districtId}&data_type=all&ak={baidu_key}"
    response = requests.get(url)
    data = response.json()
    return json.dumps(data)

In [16]:
# test tool
data = get_weather(cityName="南京")
data

'{"status": 0, "result": {"location": {"country": "\\u4e2d\\u56fd", "province": "\\u6c5f\\u82cf\\u7701", "city": "\\u5357\\u4eac\\u5e02", "name": "\\u5357\\u4eac", "id": "320100"}, "now": {"text": "\\u591a\\u4e91", "temp": 7, "feels_like": 6, "rh": 80, "wind_class": "2\\u7ea7", "wind_dir": "\\u4e1c\\u5317\\u98ce", "uptime": "20250201190000"}, "forecasts": [{"text_day": "\\u9634", "text_night": "\\u5c0f\\u96e8", "high": 9, "low": 4, "wc_day": "3~4\\u7ea7", "wd_day": "\\u5317\\u98ce", "wc_night": "3~4\\u7ea7", "wd_night": "\\u4e1c\\u5317\\u98ce", "date": "2025-02-01", "week": "\\u661f\\u671f\\u516d"}, {"text_day": "\\u9634", "text_night": "\\u591a\\u4e91", "high": 9, "low": 3, "wc_day": "<3\\u7ea7", "wd_day": "\\u5317\\u98ce", "wc_night": "<3\\u7ea7", "wd_night": "\\u5317\\u98ce", "date": "2025-02-02", "week": "\\u661f\\u671f\\u65e5"}, {"text_day": "\\u591a\\u4e91", "text_night": "\\u6674", "high": 8, "low": -1, "wc_day": "3~4\\u7ea7", "wd_day": "\\u5317\\u98ce", "wc_night": "<3\\u7ea7",

In [17]:
agent = Agent(
    name="Master Agent",
    model=checkpoint,
    instructions="You are a helpful assistant on business travel.",
    functions=[get_weather]
)

In [18]:
# 模拟场景-1
user_prompt = "帮我查下南京的天气"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [19]:
response = swarm_client.run(
    agent=agent,
    messages=messages
)

In [20]:
print(response.messages[-1]["content"])

南京今天的天气是多云，白天的温度为7℃。晚上预计有小雨夹雪，最低气温为4℃左右。明天天气持续阴天转晴冷，最高温度大约在9度。

未来几天的具体情况如下：
- 2025年2月1日（周五）白天有雨夹雪，夜间小到中雪，最低温4℃；
- 2025年2月2日（周六）与一日类似，天气阴沉，最高温度可能达到9度。
- 预计在2月3日至4日期间会经历一次冷空气带来的降温过程，会有雨夹雪转雪的天气变化。
请根据预报及时调整外出着装和行程安排。


In [28]:
# 模拟场景-2
departCityName = "南京"
arriveCityName = "深圳"
startDate = "2025-02-02"
endDate = "2025-02-10"
travelPurpose = "深圳基地培训"
query = "我明天要去深圳出差，帮我看下天气是否适合出差"

user_prompt = (
    f"出发城市：{departCityName}\n"
    f"到达城市：{arriveCityName}\n"
    f"出差时间：{startDate}至{endDate}\n"
    f"出差目的：{travelPurpose}\n"
    f"需求备注：{query}"
)

In [29]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [30]:
response = swarm_client.run(
    agent=agent,
    messages=messages
)

In [31]:
print(response.messages[-1]["content"])

根据天气预报，您计划出差期间（2月2日至2月10日）深圳的天气大部分时间是多云和小雨。其中：

- 2月2日：白天为多云，晚上转为小雨；气温较低，最高温度约为22℃，最低温度为14℃。
- 2月3日至2月5日：都是以多云为主，并且有一定概率下雨，早晚温差较大，平均在18到11摄氏度之间。

看起来大部分时间天气适合出差。但请注意随身携带雨具，保持体温舒适。如果培训行程较为户外或重要，请适当考虑穿着与保暖措施以及防护用品准备。祝您在深圳愉快，并顺利完成任务！


## step-5: Multi-Agent

In [8]:
# dialog_agent
dialog_agent = Agent(
    name="日常对话智能体（dialog_agent）",
    model=checkpoint,
    instructions=(
        "你的任务是与用户进行基本的日常对话。"
        "不要与用户分享你的思维过程！不要擅自替用户做出不合理的假设！"
    )
)

def transfer_to_dialog_agent():
    return dialog_agent

In [44]:
# weather_agent
city2districtId = {
    "北京": "110100",
    "上海": "310100",
    "广州": "440100",
    "深圳": "440300",
    "南京": "320100",
    "杭州": "330100"
}

def get_weather(city):
    """
    city: 城市名称，如 北京、南京
    """
    districtId = city2districtId.get(city)
    url = f"https://api.map.baidu.com/weather/v1/?district_id={districtId}&data_type=all&ak={baidu_key}"
    response = requests.get(url)
    data = response.json()
    return json.dumps(data)

weather_agent = Agent(
    name="查询天气智能体（weather_agent）",
    model=checkpoint,
    instructions=(
        "你的任务是调用工具帮用户查询指定城市的天气信息。"
        "不要与用户分享你的思维过程！不要擅自替用户做出不合理的假设！"
    ),
    functions=[get_weather]
)

def transfer_to_weather_agent():
    return weather_agent

In [12]:
# distance_agent
def get_distance(poiNameA, poiNameB):
    """
    poiNameA: 地标名称A
    poiNameB: 地标名称B
    """
    return "查询从A点到B点的距离与路线"

distance_agent = Agent(
    name="查询出行距离或路线智能体（distance_agent）",
    model=checkpoint,
    instructions=(
        "你的任务是调用工具帮用户查询两个地标之间的距离或路线。"
        "不要与用户分享你的思维过程！不要擅自替用户做出不合理的假设！"
    ),
    functions=[get_distance]
)

def transfer_to_distance_agent():
    return distance_agent

In [13]:
# knowledge_agent
def get_rag(query):
    """
    query: 用户的诉求
    """
    return "查询知识库回答问题"
    
knowledge_agent = Agent(
    name="差旅知识问答智能体（knowledge_agent）",
    model=checkpoint,
    instructions=(
        "你的任务是调用工具帮用户回答差旅业务知识。"
        "不要与用户分享你的思维过程！不要擅自替用户做出不合理的假设！"
    ),
    functions=[get_rag]
)

def transfer_to_knowledge_agent():
    return knowledge_agent

In [14]:
# recommend_agent
def get_hotel(query):
    """
    query: 用户的诉求
    """
    return "帮用户推荐合适的酒店"

recommend_agent = Agent(
    name="酒店推荐智能体（recommend_agent）",
    model=checkpoint,
    instructions=(
        "你的任务是调用工具帮用户推荐合适的酒店，并给出推荐的理由。"
        "不要与用户分享你的思维过程！不要擅自替用户做出不合理的假设！"
    ),
    functions=[get_hotel]
)

def transfer_to_recommend_agent():
    return recommend_agent

In [45]:
# triage_agent
intent_context = "日常对话、查询天气信息、查询出行距离或路线信息、差旅知识问答、酒店推荐"

departCityName = "南京"
arriveCityName = "深圳"
startDate = "2025-02-03"
endDate = "2025-02-10"
travelPurpose = "深圳基地培训"
customer_context = (
    f"出发城市：{departCityName}，"
    f"到达城市：{arriveCityName}，"
    f"出差时间：{startDate}至{endDate}，"
    f"出差目的：{travelPurpose}"
)

context_variables = {
    "intent_context": intent_context,
    "customer_context": customer_context
}

def triage_instructions(context_variables):
    intent_context = context_variables.get("intent_context", None)
    customer_context = context_variables.get("customer_context", None)
    instructions = (
        "任务：\n"
        "你的任务是对用户的请求进行分诊，并调用工具将请求转移到正确的意图。"
        "一旦你准备好将请求转移到正确的意图时，调用工具进行转移。"
        f"你不需要知道具体的细节，只需了解请求的主题，主题是：{intent_context}。"
        "当你需要更多信息以分诊请求至合适的智能体时，直接提出问题，而不需要解释你提问的原因。\n"
        "注意事项：\n"
        "1、不要与用户分享你的思维过程\n"
        "2、不要擅自替用户做出不合理的假设\n"
        "用户的背景信息：\n"
        f"{customer_context}。"
    )
    return instructions

triage_agent = Agent(
    name="分诊智能体",
    model=checkpoint,
    instructions=triage_instructions,
    functions=[transfer_to_dialog_agent, transfer_to_weather_agent, transfer_to_distance_agent, transfer_to_knowledge_agent, transfer_to_recommend_agent]
)

In [46]:
system_prompt = (
    "你叫小慧助手，是由Lukas开发的差旅智能客服。"
    "你的身份是一名差旅秘书，"
    "你的任务是为用户提供日常对话、查询天气、查询距离、差旅知识问答、酒店推荐等服务。"
    "当问及你的模型参数时，标准回答是属于公司保密信息，要强调模型设计的高效，能够提供高质量的服务。"
    "You are a helpful assistant on business travel."
)

In [77]:
# user_prompt = "你好"
# user_prompt = "你是什么模型？"
# user_prompt = "南京的天气"
# user_prompt = "帮我查下上海的天气"
# user_prompt = "未来天气是否适合出差？"
# user_prompt = "深圳的天气"
# user_prompt = "明天"
# user_prompt = "深圳的天气"
# user_prompt = "我的住宿标准是多少？"
# user_prompt = "帮我推荐个酒店"
# user_prompt = "吴彦祖是谁？"
# user_prompt = "我今天心情不好"
user_prompt = "和女朋友吵架了怎么办"

In [78]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [79]:
response = swarm_client.run(
    agent=triage_agent,
    messages=messages,
    context_variables=context_variables,
    stream=False
)

In [80]:
print(response.messages[-1]["content"])

这个问题虽然超出了我的专业领域，但我可以给你一些建议。遇到这种情况时，首先应该尝试冷静下来，然后找一个合适的时机和平静的心态与她沟通你的感受和观点。试着倾听她的想法，并理解事情背后的原因。寻求共同解决问题的方法而非争论对错可能是帮助你们关系改善的开始。如果需要进一步的情感咨询或者专业的建议，我建议你寻找相关方面的专家进行详细的咨询。

如果你有差旅相关的疑问或需要任何服务，请随时告诉我！
